In [1]:
import pandas as pd

# Load the CSV file
csv_file = '/content/intents_and_exampless.csv'
df_intents = pd.read_csv(csv_file)

# Display the first few rows to understand its structure
print(df_intents.head())

        Intent                    Example
0  check_human             are you a bot?
1  check_human    am i speaking to an ai?
2  check_human  are you a human or a bot?
3  check_human              are you real?
4  check_human           are you virtual?


In [2]:
import spacy
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Encode labels
label_encoder = LabelEncoder()
df_intents['Intent'] = label_encoder.fit_transform(df_intents['Intent'])

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(df_intents['Example'], df_intents['Intent'], test_size=0.2, random_state=42)

# Create a text classification pipeline
pipeline = make_pipeline(TfidfVectorizer(), SVC(probability=True))

# Train the model
pipeline.fit(X_train, y_train)

# Test the model
accuracy = pipeline.score(X_test, y_test)
print(f"Model accuracy: {accuracy * 100:.2f}%")

Model accuracy: 74.79%


In [3]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def encode_data(texts, labels, tokenizer, max_length=128):
    input_ids = []
    attention_masks = []
    for text in texts:
        encoded = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(labels)

    return input_ids, attention_masks, labels

train_inputs, train_masks, train_labels = encode_data(X_train.tolist(), y_train.tolist(), tokenizer)
test_inputs, test_masks, test_labels = encode_data(X_test.tolist(), y_test.tolist(), tokenizer)

# Create DataLoader
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, SequentialSampler

batch_size = 4

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

# Load BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
total_steps = len(train_dataloader) * 4  # Number of epochs

scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Training function
def train(model, dataloader, optimizer, scheduler, device):
    model.train()
    total_loss = 0
    for batch in dataloader:
        batch_inputs, batch_masks, batch_labels = [item.to(device) for item in batch]
        model.zero_grad()
        outputs = model(batch_inputs, token_type_ids=None, attention_mask=batch_masks, labels=batch_labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        scheduler.step()
    return total_loss / len(dataloader)

# Training loop
epochs = 3
for epoch in range(epochs):
    loss = train(model, train_dataloader, optimizer, scheduler, device)
    print(f"Epoch {epoch+1}/{epochs} - Loss: {loss:.4f}")

# Evaluation function
def evaluate(model, dataloader, device):
    model.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0
    for batch in dataloader:
        batch_inputs, batch_masks, batch_labels = [item.to(device) for item in batch]
        with torch.no_grad():
            outputs = model(batch_inputs, token_type_ids=None, attention_mask=batch_masks, labels=batch_labels)
            loss = outputs.loss
            logits = outputs.logits
            total_eval_loss += loss.item()
            preds = torch.argmax(logits, dim=1).flatten()
            accuracy = (preds == batch_labels).cpu().numpy().mean() * 100
            total_eval_accuracy += accuracy
    return total_eval_loss / len(dataloader), total_eval_accuracy / len(dataloader)

# Evaluate the model
loss, accuracy = evaluate(model, test_dataloader, device)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.2f}%")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/3 - Loss: 2.6552
Epoch 2/3 - Loss: 1.8498
Epoch 3/3 - Loss: 1.3599
Test Loss: 1.4119, Test Accuracy: 76.39%


In [4]:
import pandas as pd

# Load the new dataset
new_csv_file = '/content/test-customer-dataset.csv'
df_customers = pd.read_csv(new_csv_file)

# Display the first few rows to understand its structure
print(df_customers.head())

   RowNumber  CustomerId   Surname  CreditScore Geography  Gender   Age  \
0          1    15634602  Hargrave          619    France  Female  42.0   
1          2    15647311      Hill          608     Spain  Female  41.0   
2          3    15619304      Onio          502    France  Female  42.0   
3          4    15701354      Boni          699    France  Female  39.0   
4          5    15737888  Mitchell          850     Spain  Female  43.0   

   Tenure    Balance  NumOfProducts  HasCrCard  IsActiveMember  \
0       2       0.00              1        1.0             1.0   
1       1   83807.86              1        0.0             1.0   
2       8  159660.80              3        1.0             0.0   
3       1       0.00              2        0.0             0.0   
4       2  125510.82              1        NaN             1.0   

   EstimatedSalary  Exited  
0        101348.88       1  
1        112542.58       0  
2        113931.57       1  
3         93826.63       0  
4      

In [5]:
import os
import sqlite3
import pandas as pd
import sqlite3

# Read the CSV file
csv_file = '/content/test-customer-dataset.csv'
df = pd.read_csv(csv_file)


# Create a connection to the SQLite database
conn = sqlite3.connect('chatbot.db')
cursor = conn.cursor()

# Convert the DataFrame to a SQL table
df_customers.to_sql('customers', conn, if_exists='replace', index=False)

# Verify by querying the database
cursor.execute("SELECT * FROM customers LIMIT 5")
rows = cursor.fetchall()
for row in rows:
    print(row)

# Commit and close the connection
conn.commit()
conn.close()

(1, 15634602, 'Hargrave', 619, 'France', 'Female', 42.0, 2, 0.0, 1, 1.0, 1.0, 101348.88, 1)
(2, 15647311, 'Hill', 608, 'Spain', 'Female', 41.0, 1, 83807.86, 1, 0.0, 1.0, 112542.58, 0)
(3, 15619304, 'Onio', 502, 'France', 'Female', 42.0, 8, 159660.8, 3, 1.0, 0.0, 113931.57, 1)
(4, 15701354, 'Boni', 699, 'France', 'Female', 39.0, 1, 0.0, 2, 0.0, 0.0, 93826.63, 0)
(5, 15737888, 'Mitchell', 850, 'Spain', 'Female', 43.0, 2, 125510.82, 1, None, 1.0, 79084.1, 0)


In [28]:
import sqlite3

def fetch_customer_balance(Surname):
    conn = sqlite3.connect('chatbot.db')
    c = conn.cursor()
    c.execute('SELECT Balance FROM customers WHERE Surname = ?', (Surname,))
    result = c.fetchone()
    conn.close()
    if result:
        return f"Your current balance is ${result[0]}."
    else:
        return "No balance data available."

def transfer_money(name, receiver_name, amount):
    conn = sqlite3.connect('chatbot.db')
    c = conn.cursor()

    # Fetch sender's balance by surname
    c.execute('SELECT Balance FROM customers WHERE Surname = ?', (name,))
    sender_balance = c.fetchone()

    # Fetch receiver's balance by surname
    c.execute('SELECT Balance FROM customers WHERE Surname = ?', (receiver_name,))
    receiver_balance = c.fetchone()

    if not receiver_balance:
        conn.close()
        return f"Receiver with Name {receiver_name} not found."
    if not sender_balance or sender_balance[0] < amount:
        conn.close()
        return f"Insufficient funds."

    try:
        # Begin transaction
        print(f"Sender's balance before transfer: ${sender_balance[0]}")
        print(f"Receiver's balance before transfer: ${receiver_balance[0]}")
        conn.execute('BEGIN TRANSACTION')

        # Decrease balance of sender
        c.execute('UPDATE customers SET Balance = Balance - ? WHERE Surname = ?', (amount, name))

        # Increase balance of receiver
        c.execute('UPDATE customers SET Balance = Balance + ? WHERE Surname = ?', (amount, receiver_name))

        # Commit transaction
        conn.commit()

        c.execute('SELECT Balance FROM customers WHERE Surname = ?', (name,))
        updated_sender_balance = c.fetchone()

        c.execute('SELECT Balance FROM customers WHERE Surname = ?', (receiver_name,))
        updated_receiver_balance = c.fetchone()

        # Print balances after the transaction
        print(f"Sender's balance after transfer: ${updated_sender_balance[0]}")
        print(f"Receiver's balance after transfer: ${updated_receiver_balance[0]}")


        conn.close()
        return f"Successfully transferred ${amount} to {receiver_name}."
    except sqlite3.Error as e:
        # Rollback transaction in case of error
        conn.rollback()
        conn.close()
        return f"Failed to transfer money due to: {str(e)}"

In [29]:
import re
session_data = {}
def classify_intent(query):
      inputs = tokenizer.encode_plus(
        query,
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )


      input_ids = inputs['input_ids'].to(device)
      attention_mask = inputs['attention_mask'].to(device)

      with torch.no_grad():
          outputs = model(input_ids, token_type_ids=None, attention_mask=attention_mask)

      logits = outputs.logits
      intent = torch.argmax(logits, dim=1).cpu().numpy()[0]

      classified_intent = label_encoder.inverse_transform([intent])[0]

    # Print statement to check the classified intent
      print(f"Query: '{query}' classified as Intent: '{classified_intent}'")

      return classified_intent

def extract_entities(text):
    doc = nlp(text)
    entities = {ent.label_: ent.text for ent in doc.ents}

    # Use regex to capture names if not recognized by SpaCy
    if "PERSON" not in entities:
        # Look for a pattern like "to [Name]"
        match = re.search(r'\bto\s+([A-Z][a-z]+)\b', text)
        if match:
            entities["PERSON"] = match.group(1)
    return entities



def generate_response(user_input, name):
    global session_data
    intent = classify_intent(user_input)
    entities = extract_entities(user_input)

    # Update session data with new entities
    session_data.update(entities)

    if intent == "check_balance":
        return fetch_customer_balance(name)
    elif intent == "transfer_money":
        amount = session_data.get("MONEY", None)
        receiver_name = session_data.get("PERSON", None)

        if amount and receiver_name:
            # Clear the session data after use
            session_data.clear()
            amount = float(amount.replace('$', '').replace(',', ''))
            return transfer_money(name, receiver_name, amount)
        else:
            return "Please provide receiver's name and amount to transfer."
    elif intent == "check_human":
        return "I am an AI created to assist you with financial inquiries."
    elif intent == 'open_account':
        return "To open a new account, please visit our website or nearest branch with your identification documents."
    elif intent == 'close_account':
        return "To close your account, please visit our nearest branch or contact our customer support."
    elif intent == "loan_inquiry":
        return "You can inquire about loans and their eligibility criteria on our website or by visiting our branch."
    elif intent == 'credit_card_application':
        return "You can apply for a credit card through our website or by visiting our nearest branch."
    elif intent == 'contact_support':
        return "You can contact our customer support through the chat feature on our website or by calling our support number."
    else:
        return "Sorry, I don't understand your query."

def chat():
    print("Welcome to the Financial Chatbot! How can I assist you today?")
    name = input("Please enter your name: ")
    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit", "bye"]:
            print("Chatbot: Thank you for using the Financial Chatbot. Have a great day!")
            break
        response = generate_response(user_input, name)
        print(f"Chatbot: {response}")

if __name__ == "__main__":
    chat()


Welcome to the Financial Chatbot! How can I assist you today?
Please enter your name: Romeo
You: What is my account balance
Query: 'What is my account balance' classified as Intent: 'check_balance'
Extracted entities: {}
Chatbot: Your current balance is $132502.88.
You: How much money is on my account
Query: 'How much money is on my account' classified as Intent: 'check_balance'
Extracted entities: {}
Chatbot: Your current balance is $132502.88.
You: Are you a human
Query: 'Are you a human' classified as Intent: 'check_human'
Extracted entities: {}
Chatbot: I am an AI created to assist you with financial inquiries.
You: I want to transfer $100 to Chu
Query: 'I want to transfer $100 to Chu' classified as Intent: 'transfer_money'
Extracted entities: {'MONEY': '100', 'PERSON': 'Chu'}
Sender's balance before transfer: $132502.88
Receiver's balance before transfer: $113755.78
Sender's balance after transfer: $132402.88
Receiver's balance after transfer: $113855.78
Chatbot: Successfully tran